<a href="https://colab.research.google.com/github/jennifer060697/AI15_Project/blob/main/n212_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*AIB/SECTION2/SPRINT1/NOTE2*
============================

------------------------------


Generalization